### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
from IPython.core.display import display, HTML, clear_output
from ipywidgets import Button, Text, Layout, Box, Output
import functools

### Выгрузка данных
Cсылка на API: https://github.com/hhru/api 

In [2]:
# Выбор названий, временного отрезка и опыта

text_list = [
            'Data Scientist',
            'Data Science',
            'Дата Cаентист',
            'Дата Саенс', 
            'Machine Learning',
            'Машинное обучение',
            'ML',
#             'Специалист по machine learning',
#             'Специалист по машинному обучению',
#             'Machine Learning Engineer',
#             'Machine Learning Developer',
#             'Machine Learning Researcher',
#             'Разработчик Machine Learning',
#             'Machine Learning инженер',
#             'ML Engineer',
#             'ML Developer',
#             'ML Researcher',
#             'Разработчик ML',
#             'ML инженер',
            'Data Analyst'
            'Аналитик данных',
            'Специалист по анализу данных'
            ]

text_str = '" or "'.join([i for i in text_list])
text = f'SQL NAME:("{text_str}")'

dt_now = datetime.now().date()
dt_from = dt_now - timedelta(days=7)
date_from = '%s-%s-%s' % (dt_from.year, dt_from.month, dt_from.day)

url_0 = 'https://api.hh.ru/'
url_1 = url_0 + f'vacancies?text={text}&date_from={date_from}&responses_count_enabled=True&per_page=100'

experiences = ['noExperience', 'between1And3']

In [3]:
# Выгрузка данных

request_data = pd.DataFrame()
for exp in experiences:
    page = 0
    url_2 = url_1 + f'&experience={exp}'
    try:
        r = requests.get(url_2 + f'&page={page}')
        data = json.loads(r.text)
        print(exp, ' - ', data['pages'], 'pages')
        items = data['items']
        for page in range(1, data['pages']):
            r = requests.get(url_2 + f'&page={page}')
            data = json.loads(r.text)
            items += data['items']
        df = pd.DataFrame(items)
        df['experience'] = exp
        request_data = pd.concat([request_data, df])
    except Exception as e:
        print(exp, '-', e)
        
request_data.reset_index(drop=True, inplace=True)
print('\nDataFrame shape:', request_data.shape)
pd.set_option('display.max_columns', None);
request_data.sample(3)

noExperience  -  1 pages
between1And3  -  2 pages

DataFrame shape: (113, 30)


,id,premium,name,department,has_test,response_letter_required,area,salary,type,address,response_url,sort_point_distance,published_at,created_at,archived,apply_alternate_url,insider_interview,url,alternate_url,relations,employer,snippet,contacts,schedule,counters,working_days,working_time_intervals,working_time_modes,accept_temporary,experience
73,46125724,False,Middle data scientist,"{'id': '2343-2343-it', 'name': 'Компания «СПОР...",False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': None, 'street': None, 'building': Non...",None,None,2021-10-29T12:02:13+0300,2021-10-29T12:02:13+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/46125724?host=hh.ru,https://hh.ru/vacancy/46125724,[],"{'id': '2343', 'name': 'Спортмастер', 'url': '...",{'requirement': 'Аналитический подход к решени...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 1},[],[],[],False,between1And3
97,48524305,False,Data scientist / Специалист по нейросетям,None,False,False,"{'id': '2089', 'name': 'Ивантеевка', 'url': 'h...",None,"{'id': 'open', 'name': 'Открытая'}",None,None,None,2021-10-29T17:09:36+0300,2021-10-29T17:09:36+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/48524305?host=hh.ru,https://hh.ru/vacancy/48524305,[],"{'id': '962687', 'name': 'АНТРАКС, МНПП', 'url...",{'requirement': 'Знание <highlighttext>SQL</hi...,None,"{'id': 'flexible', 'name': 'Гибкий график'}",{'responses': 0},[],[],[],False,between1And3
21,49145826,False,Machine Learning Developer,None,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'улица М...",None,None,2021-10-29T11:55:31+0300,2021-10-29T11:55:31+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/49145826?host=hh.ru,https://hh.ru/vacancy/49145826,[],"{'id': '1035551', 'name': 'WebMasters Russia',...",{'requirement': 'Опыт коммерческой разработки ...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 3},[],[],[],False,between1And3


### Фильтрация датасета

In [4]:
# Извлечение нужных столбцов и информации из словарей в новый датафрейм

def return_id(x, key='id'):
    try:
        return x[key]
    except:
        return np.nan
    
vacancies = request_data.copy()[['id', 'name', 'published_at', 'url', 'alternate_url', 'experience']]

vacancies['employer'] = request_data['employer'].apply(return_id, key='name')
vacancies['city'] = request_data['area'].apply(return_id, key='name')
vacancies['type'] = request_data['type'].apply(return_id, key='name')
vacancies['schedule'] = request_data['schedule'].apply(return_id, key='name')
vacancies['responses'] = request_data['counters'].apply(return_id, key='responses')
vacancies['salary_from'] = request_data['salary'].apply(return_id, key='from')
vacancies['salary_to'] = request_data['salary'].apply(return_id, key='to')
vacancies['currency'] = request_data['salary'].apply(return_id, key='currency')

In [5]:
# Фильтрация датасета

black_words = ['ведущий', 'старший', 'senior', 'middle', 'teamlead', 'team lead',  'lead', 'руководитель', 
               'мидл', 'миддл', 'сеньор', 'сениор', 'тимлид', 'тим лид',  'главный', 'директор', 'управляющий']
white_words = ['junior', 'juniour', 'джуниор', 'младший', 'стажер', 'стажёр', 'интерн', 'intern']

def check_name(col):
    return (any([i in col.lower() for i in black_words]) &
            (not any([i in col.lower() for i in white_words])))

mask = (
            (
                (vacancies['city']!='Москва') & 
                (vacancies['city']!='Нижний Новгород') & 
                (vacancies['schedule']!='Удаленная работа')
            ) 
            |
            (
                vacancies['type']!='Открытая'
            ) 
            |
            (
                vacancies['name'].apply(check_name)
            )
       )

vacancies.drop(labels=vacancies[mask].index, inplace=True)
vacancies.drop_duplicates(subset=['name', 'employer'], inplace=True)
vacancies.sort_values('published_at', ascending=False, inplace=True)
vacancies.reset_index(drop=True, inplace=True)

print('DataFrame shape:', vacancies.shape)
vacancies.head(5)

DataFrame shape: (48, 14)


,id,name,published_at,url,alternate_url,experience,employer,city,type,schedule,responses,salary_from,salary_to,currency
0,49277183,Бизнес-аналитик (Data Science направление),2021-11-01T10:30:20+0300,https://api.hh.ru/vacancies/49277183?host=hh.ru,https://hh.ru/vacancy/49277183,between1And3,red_mad_robot,Москва,Открытая,Полный день,0,NaN,NaN,NaN
1,48980498,Data scientist,2021-11-01T09:44:32+0300,https://api.hh.ru/vacancies/48980498?host=hh.ru,https://hh.ru/vacancy/48980498,between1And3,"МАГНИТ, Розничная сеть",Москва,Открытая,Удаленная работа,0,NaN,NaN,NaN
2,49172940,Data scientist (CRM),2021-11-01T09:26:10+0300,https://api.hh.ru/vacancies/49172940?host=hh.ru,https://hh.ru/vacancy/49172940,between1And3,"РОСБАНК, Societe Generale Group (Russia)",Москва,Открытая,Полный день,0,NaN,NaN,NaN
3,49218764,Data Scientist (клиентские данные),2021-11-01T09:26:05+0300,https://api.hh.ru/vacancies/49218764?host=hh.ru,https://hh.ru/vacancy/49218764,between1And3,"РОСБАНК, Societe Generale Group (Russia)",Москва,Открытая,Полный день,2,NaN,NaN,NaN
4,48500730,Data Scientist,2021-10-31T17:32:33+0300,https://api.hh.ru/vacancies/48500730?host=hh.ru,https://hh.ru/vacancy/48500730,between1And3,Plarium,Москва,Открытая,Полный день,46,NaN,NaN,NaN
5,48434078,Data Scientist,2021-10-31T12:06:52+0300,https://api.hh.ru/vacancies/48434078?host=hh.ru,https://hh.ru/vacancy/48434078,between1And3,CORE,Москва,Открытая,Удаленная работа,487,180000.0,250000.0,RUR
6,49213609,Machine Learning Engineer,2021-10-31T12:05:56+0300,https://api.hh.ru/vacancies/49213609?host=hh.ru,https://hh.ru/vacancy/49213609,between1And3,Nexus FrontierTech Ltd,Москва,Открытая,Удаленная работа,11,200000.0,350000.0,RUR
7,46639991,Аналитик данных / Data scientist,2021-10-30T20:20:28+0300,https://api.hh.ru/vacancies/46639991?host=hh.ru,https://hh.ru/vacancy/46639991,between1And3,Лендсбэй,Москва,Открытая,Полный день,10,NaN,NaN,NaN
8,49044672,Junior Data Scientist (Специалист по анализу д...,2021-10-30T15:41:13+0300,https://api.hh.ru/vacancies/49044672?host=hh.ru,https://hh.ru/vacancy/49044672,noExperience,АлгоМост,Москва,Открытая,Полный день,142,NaN,NaN,NaN
9,48139130,Product Data Science проектов,2021-10-30T15:22:42+0300,https://api.hh.ru/vacancies/48139130?host=hh.ru,https://hh.ru/vacancy/48139130,between1And3,Level.Travel,Москва,Открытая,Гибкий график,21,NaN,250000.0,RUR


### Выгрузка и просмотр содержимого

In [6]:
# Выгрузка содержимого

vacs = vacancies.head(50)['url'].apply(lambda x: HTML(json.loads(requests.get(x).text)['description']))

In [19]:
# Настройки панели управления

items = [Button(layout=Layout(width='47.5%'), icon='arrow-left'),
         Text(value='', layout=Layout(width='5%')),
         Button(layout=Layout(width='47.5%'), icon='arrow-right')]
box_layout = Layout(display='flex', flex_flow='row', align_items='stretch', width='100%')
box = Box(children=items, layout=box_layout)
output = Output()
display(box, output)

# Функция для печати содержимого
def print_content(idx):
    clear_output()
    display(HTML('<a href="' + str(vacancies['alternate_url'][idx]) + '" target="_blank">Ссылка на вакансию</a>'))
    print('Опубликована:', vacancies['published_at'][idx][:10], ' Отправлено откликов:', vacancies['responses'][idx])
    display(HTML("<h3>" + str(vacancies['name'][idx]) + ' в ' + str(vacancies['employer'][idx]) + "</h3>"))
    display(vacs[idx])

# Функция для реакции на клик
def on_clicked(b, direction=0):
    with output:
        try:
            global vac_idx
            try:
                vac_idx = (vac_idx + direction)%len(vacs)
            except:
                vac_idx = 0
            box.children[1].value = str(vac_idx)
            print_content(vac_idx)
        except Exception as e:
            clear_output()
            print(e)
    
# Функция для реакции на ввод текста
def on_submitted(b):
    with output:
        try:
            global vac_idx
            val = int(box.children[1].value)
            if val >= len(vacs):
                vac_idx = len(vacs) - 1
            elif val < 0:
                vac_idx = 0
            else:
                vac_idx = val
            box.children[1].value = str(vac_idx)
            print_content(vac_idx)
        except Exception as e:
            clear_output()
            print(e)

box.children[0].on_click(functools.partial(on_clicked, direction=-1))
box.children[1].on_submit(on_submitted)
box.children[2].on_click(functools.partial(on_clicked, direction=1))

Box(children=(Button(icon='arrow-left', layout=Layout(width='47.5%'), style=ButtonStyle()), Text(value='', lay…

Output()

In [8]:
#from bs4 import BeautifulSoup
#text = vacs_df['url'].apply(lambda x: BeautifulSoup(json.loads(requests.get(x).text)['description']).get_text())
#text[18]